# RNNを用いた言語モデル

RNNを用いて言語モデルを実装する。

前章では、深層学習を活用してある単語から次の単語を予測するモデルを作成した。しかし、これでは全体の文脈全体を考慮できない。具体的には、2つ以上前の単語を考慮した予測が出来ない。

RNNを用いることで、文脈を考慮した予測が可能になる。  
本章ではRNNの構造について学び、RNNを用いた言語モデルを実装する。

In [1]:
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(width=20, with_test=True, label="ppl train", round=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 学習データの用意

In [3]:
n_data = 20000

wiki40b

In [4]:
text_path = "data/jawiki.txt"
with open(text_path) as f:
    data = f.read().splitlines()

In [5]:
data = data[:n_data]
text_path = f"data/jawiki_{n_data}.txt"
with open(text_path, 'w') as f:
    f.write("\n".join(data))

print("num of data:", len(data))
data[:3] # examples

num of data: 20000


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']


---

## RNN

*Recurrent Neural Network*

再帰型ニューラルネットワーク。  
再帰的な構造を持つニューラルネットワークで、可変長の時系列データを扱うことが得意。

RNNはある時間$t$の入力$x_t$に対して以下のような演算を行い、出力値$h_t$を決定する。

$$
h_t = \text{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

演算の内部で前の時間の出力値$h_{t-1}$を参照していることが分かる。  
演算内容はシンプルに捉えることができて、入力$x_t$と前の時間の出力$h_{t-1}$をそれぞれ線形変換し、それらの和を活性化関数$\text{tanh}$に通しているだけ。線形変換に必要な重みとバイアスが2つずつあるので、パラメータは合計4つ。  
活性化関数は$\text{tanh}$でなくてもよいが、大体は$\text{tanh}$が使われる。

実装してみるとこんな感じ。

In [6]:
class RNNCell(nn.Module):
    def __init__(self, input_size: int, output_size: int):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_output = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        """
            x: (batch_size, input_size)
            h: (batch_size, output_size)
        """
        z = self.fc_input(x) + self.fc_output(h) # (batch_size, output_size)
        h = F.tanh(z)
        return h

推論の流れを見てみる。  
まず適当なパラメータで出力の初期値$h_0$と初めの入力$x_1$を定義する。$h_0$は0ベクトルで良い。

In [7]:
batch_size, input_size, hidden_size = 2, 3, 4
rnn = RNNCell(input_size, hidden_size)

x1 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, hidden_size)

h1 = rnn(x1, h0)
h1

tensor([[-0.6300, -0.4942,  0.2800, -0.0061],
        [ 0.6367,  0.4952, -0.0675,  0.8148]], grad_fn=<TanhBackward0>)

この$h_1$を参照して、次の時間の入力$x_2$に対する出力$h_2$を決定する。

In [8]:
x2 = torch.randn(batch_size, input_size)
h2 = rnn(x2, h1)
h2

tensor([[-0.5272, -0.0823,  0.5397,  0.4693],
        [ 0.9065,  0.3946, -0.3703,  0.4223]], grad_fn=<TanhBackward0>)

これがRNNの推論の流れである。  
この一連の流れをまとめたモデルを実装する。

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)

    def forward(self, x, h):
        """
            x: (seq_len, batch_size, input_size)
            h: (batch_size, hidden_size)
        """
        hs = []
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        return hs

任意の長さの入力に対して、同じ数の隠れ状態を出力する。

In [10]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(seq_len, batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([5, 2, 4])

少し扱いやすくしたいので書き換える。以下を実現する。
- 系列長の次元ではなくバッチ次元を初めにした形状`(batch_size, seq_len, hidden_size)`で入出力を扱えるようにする。
- 隠れ状態`h`の入力がなかった場合に自動で0ベクトルになるようにする。

In [11]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    def forward(self, x, h=None):
        """
            x: (batch_size, seq_len, input_size)
            h: (batch_size, hidden_size)
        """
        if h is None:
            h = torch.zeros(x.size(0), self.hidden_size).to(x.device)
        hs = []
        x = x.transpose(0, 1) # (seq_len, batch_size, input_size)
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        hs = hs.transpose(0, 1) # (batch_size, seq_len, hidden_size)
        return hs

`transpose(0, 1)`でバッチ次元と系列長の次元を入れ替える事で実現した。

In [12]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(batch_size, seq_len, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([2, 5, 4])

### PyTorchでの実装

PyTorchにはRNNを実装するためのクラスが用意されている。軽く紹介する。

#### `torch.nn.RNNCell`

RNNの一つの時間の演算を行うクラス。初めに実装したもの。  
https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html

In [13]:
batch_size, input_size, hidden_size = 2, 3, 4

rnn_cell = nn.RNNCell(input_size, hidden_size)
for params in rnn.parameters():
    print(params, "\n")

Parameter containing:
tensor([[-0.2240,  0.3780, -0.2061],
        [-0.3323,  0.1513,  0.1095],
        [ 0.2460,  0.4345, -0.2894],
        [ 0.0140,  0.0751, -0.3647]], requires_grad=True) 

Parameter containing:
tensor([ 0.0818,  0.4554,  0.0239, -0.5003], requires_grad=True) 

Parameter containing:
tensor([[ 4.4125e-01,  2.6982e-01,  4.6929e-01,  4.9711e-01],
        [ 4.6931e-01, -3.3048e-01, -4.8511e-01,  3.0277e-01],
        [-3.1799e-01,  4.9294e-01,  2.4760e-04,  1.0926e-01],
        [-3.5755e-01, -1.3702e-01,  1.4277e-01,  9.8905e-02]],
       requires_grad=True) 

Parameter containing:
tensor([-0.3000,  0.2519, -0.2965,  0.1009], requires_grad=True) 



ちゃんとパラメータが4つあるね。

推論も同じ。

In [14]:
x = torch.randn(batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)
rnn_cell(x, h)

tensor([[-0.4783, -0.8784,  0.2163, -0.7640],
        [ 0.0084, -0.7361, -0.3742, -0.0690]], grad_fn=<TanhBackward0>)

ちなみに$h$は入力しなくてもいい。その場合は勝手に0ベクトルになる。

In [15]:
rnn_cell(x)

tensor([[-0.4783, -0.8784,  0.2163, -0.7640],
        [ 0.0084, -0.7361, -0.3742, -0.0690]], grad_fn=<TanhBackward0>)

#### `torch.nn.RNN`

全ての時間の演算をまとめて行うクラス。  
https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [16]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

x = torch.randn(batch_size, seq_len, input_size)
hs, h = rnn(x)
hs.shape

torch.Size([2, 5, 4])

全ての時間の隠れ状態が出力される。`batch_first=True`とすることでバッチ次元を初めできる。

最後の時間の隠れ状態も別で出力される。

In [17]:
hs[:, -1] == h

tensor([[[True, True, True, True],
         [True, True, True, True]]])

In [18]:
h.shape

torch.Size([1, 2, 4])

初めに次元が1つ追加されるのは仕様。


---

## RNNを用いた言語モデル

RNNを用いた言語モデル構築の流れを見てみる。

基本的に、RNNそのものを1つのモデルとして扱うことはない。RNNはNNの中の1つの層として扱う。  
なお、RNNが組み込まれたモデルをRNNと呼ぶこともある。混乱を避けるために、本章では層としてのRNNはRNN層と呼ぶこととする（モデルとしてのRNNはそのままRNNと呼ぶ）。

ある時間$t$におけるRNN層は、その時間の入力$x_t$と前の時間のRNN層の出力$h_{t-1}$を受け取り、$h_t$を出力する。  
$h_t$は外から直接観測されることがないため、**隠れ状態**（や潜在変数）と表現される。変数名に$h$を使っている理由がそこにある（隠れる=hide）。

では、RNN層をNNに取り入れて言語モデルを作ろう。以下のような最低限のネットワークを構築する。

In [19]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = RNN(embed_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (1, batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        hs = self.rnn(x, h) # hs: (batch_size, seq_length, hidden_size)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        return y, hs

このモデルで再帰的な推論を行うことで文章生成が可能になる。初めの単語$w_0$（`bos`）と隠れ状態$h_0$（0ベクトル）を入力し、次の単語$w_1$と隠れ状態$h_1$を得る。次に$w_1$と$h_1$を再度モデルに入力し、$w_2$と$h_2$を得る。これを繰り返すことで、文脈を考慮した文章生成を実現する（この詳細は後のBPTTの章にて説明する）。

流れを見てみよう。

In [20]:
n_vocab, embed_dim, hidden_size = 8000, 512, 512
model = LanguageModel(n_vocab, embed_dim, hidden_size)

テストなので、語彙数は少なめに10としている。  
次に初めの単語$w_0$を用意する。

In [21]:
w0 = torch.tensor(0).reshape(1, -1) # (batch_size, seq_length) = (1, 1)
w0

tensor([[0]])

初めの隠れ状態$h_0$も用意し、これらをモデルに入力する。

In [22]:
h0 = torch.zeros(1, hidden_size)
y1, h1 = model(w0, h0)
y1.shape, h1.shape

(torch.Size([1, 1, 8000]), torch.Size([1, 1, 512]))

出力を確率分布に変換し、次の単語をサンプリングする。

In [23]:
dist = F.softmax(y1.squeeze(), dim=-1)
w1, = random.choices(range(n_vocab), weights=dist)
w1

2407

これを繰り返すことで文章を生成する。


---

## 学習データの作成

モデルの学習に使用する、入力と出力のペアを作成する。どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。この3つの単語の誤差とその勾配は一度に求められる。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。

In [24]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __getitem__(self, idx):
        text = self.data[idx]
        in_text = text[:-1] # 入力単語列
        out_text = text[1:] # 入力を1つずつずらした単語列
        return in_text, out_text

    def __len__(self):
        return self.n_data


---

## BPTT

*Back Propagation Through Time*

時間を跨いだ逆伝播。

RNNでは、ある時間$t$における演算で、その時間の入力$x_t$以外に、隠れ状態（前のRNN層からの出力）$h_{t-1}$を参照する。  
これは$x_t$のみを参照する通常のNNとは大きく異なる点である。

言語モデルの様な、各時間で確率分布からのサンプリングを挟むようなモデルでは、この隠れ状態が重要な役割を果たす。それは逆伝播が時間を跨いで繋がる点である。これをBPTT (*Back Propagation Through Time*) と呼ぶ。

BPTTによって時間を遡って勾配を届けられるようになり、長期的な視点で誤差が減るように学習できる。その際、勾配の伝達を担う隠れ状態には、長期的な文脈の情報が載るようになることが期待される。

ちなみに、サンプリングを挟まない場合は普通に逆伝播が届く。モデルの出力値をそのまま入力する場合とか。  
あとは、サンプリングといっても、出力されたデータの一部を次に入力するような場合も普通に逆伝播が届く。

### Truncated BPTT

BPTTで細かく時間を区切る手法。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなる。ただ、実はRNNの時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

実装は以下のようになる。

```python
trunc_len = 64 # 区切る長さ

for x, t in dataloader:
    h = None # 隠れ状態の初期化

    # Truncated BPTT
    for i in range(0, x.shape[1], trunc_len):
        x_batch = x[:, i:i+trunc_len].to(device)
        t_batch = t[:, i:i+trunc_len].to(device)

        optimizer.zero_grad()
        y, h = model(x_batch, h)
        loss = loss_fn(y, t_batch)
        loss.backward()
        optimizer.step()

        h = h.detach() # 計算グラフから切り離す
```

あくまでも区切るのは逆伝播だけなので、順伝播時の隠れ状態はちゃんと保持される。


---

## ミニバッチ学習

RNNを用いた言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [25]:
from torch.nn.utils.rnn import pad_sequence

In [26]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。  
まずpadトークンを入れたトークナイザを作る。

In [ ]:
pad_id = 3
vocab_size = 8000
tokenizer_prefix = f"models/tokenizer_jawiki_{n_data}"

spm.SentencePieceTrainer.Train(
    input=text_path,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

これでトークン化。

In [28]:
sp = spm.SentencePieceProcessor(f"{tokenizer_prefix}.model")
n_vocab = len(sp)

unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()

data_ids = sp.encode(data)
for ids in data_ids:
    ids.insert(0, bos_id)
    ids.append(eos_id)

n_vocab = len(sp)
print("num of vocabrary:", n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[1, 12, 20, 528, 495, 283, 48, 558, 52, 3542]

そんで、バッチ内データが揃う用にDataLoaderを作成する。

In [29]:
def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 64
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

sample_x, sample_y = next(iter(train_loader))
print("input batch shape", sample_x.shape)
sample_x[0][:10], sample_y[0][:10] # example

input batch shape torch.Size([64, 1766])


(tensor([   1,   12,  356,   11,  114,   17, 2416,   11,   16,    5]),
 tensor([  12,  356,   11,  114,   17, 2416,   11,   16,    5, 3177]))

これで学習データはOK。

最後に、学習時に使う交差エントロピーでパディング用のトークンを無視するように定義すれば完了。

In [30]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)


---

## 実践

実際にモデルを作って文章を生成してみよう。

先程作成したモデルを少しいじる。

In [31]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = RNN(embed_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        x = self.dropout(x)
        hs = self.rnn(x, h) # (batch_size, seq_length, hidden_size)
        h = hs[:, -1] # (batch_size, hidden_size)
        hs = self.dropout(hs)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        return y, h

ドロップアウト層を追加した。

In [32]:
n_vocab = len(sp)
embed_dim = 512
hidden_size = 512
model = LanguageModel(n_vocab, embed_dim, hidden_size).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 8,725,312


### 学習

学習させる。

In [33]:
def loss_fn(y, t):
    """
    y: (batch_size, seq_length, n_vocab)
    t: (batch_size, seq_length)
    """
    loss = cross_entropy(y.reshape(-1, n_vocab), t.ravel())
    return loss


def eval_model(model, trunc_len=100):
    model.eval()
    losses = []
    with torch.no_grad():
        for x, t in test_loader:
            h = None
            for i in range(0, x.shape[1], trunc_len):
                x_batch = x[:, i:i+trunc_len].to(device)
                t_batch = t[:, i:i+trunc_len].to(device)
                y, h = model(x_batch, h)
                loss = loss_fn(y, t_batch)
                losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = torch.exp(torch.tensor(loss)).item()
    return ppl


def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    # trunc_len: Truncated BPTTで区切る長さ

    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            h = None

            # Truncated BPTT
            for i in range(0, x.shape[1], trunc_len):
                trunc_x = x[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len, n_vocab)
                trunc_t = t[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len)

                optimizer.zero_grad()
                y, h = model(trunc_x, h)
                loss = loss_fn(y, trunc_t)
                loss.backward()
                optimizer.step()
                ppl = torch.exp(loss).item()
                prog.update(ppl, advance=0)
                h = h.detach() # 計算グラフから切り離す
            prog.update()

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [34]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [35]:
train(model, optimizer, trunc_len=100, n_epochs=40, prog_unit=2)

  1-2/40: #################### 100% [00:05:40.26] ppl train: 868.36, test: 423.20 
  3-4/40: #################### 100% [00:05:35.20] ppl train: 387.09, test: 297.86 
  5-6/40: #################### 100% [00:05:38.53] ppl train: 277.48, test: 245.77 
  7-8/40: #################### 100% [00:05:37.52] ppl train: 227.07, test: 210.50 
 9-10/40: #################### 100% [00:05:37.71] ppl train: 197.64, test: 190.66 
11-12/40: #################### 100% [00:05:41.22] ppl train: 177.61, test: 180.67 
13-14/40: #################### 100% [00:05:39.99] ppl train: 164.43, test: 175.61 
15-16/40: #################### 100% [00:05:38.78] ppl train: 154.14, test: 167.59 
17-18/40: #################### 100% [00:05:39.22] ppl train: 145.50, test: 162.39 
19-20/40: #################### 100% [00:05:39.77] ppl train: 138.55, test: 159.69 
21-22/40: #################### 100% [00:05:40.38] ppl train: 133.22, test: 162.75 
23-24/40: #################### 100% [00:05:38.56] ppl train: 129.19, test: 157.23 
25-2

In [36]:
model_path = "models/lm_rnn.pth"
torch.save(model.state_dict(), model_path)

### 文章生成

学習したモデルを使って文章を生成する。  
モデルが出力した単語を次の入力とし、再帰的に出力を得る。

In [37]:
model = LanguageModel(n_vocab, embed_dim, hidden_size).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [41]:
def token_sampling(y):
    y = y.squeeze()
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = "",
    max_len: int = 100
) -> str:
    model.eval()

    # 隠れ状態の初期値
    token_ids = sp.encode(start)
    token_ids.insert(0, bos_id)
    x = torch.tensor(token_ids, device=device).unsqueeze(0)
    y, hs = model(x)
    h = hs[:, -1]
    next_token = token_sampling(y)
    token_ids.append(next_token)

    # 続きの文章生成
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).unsqueeze(0)
        y, (h,) = model(x, h)
        next_token = token_sampling(y)
        token_ids.append(next_token)

    sentence = sp.decode(token_ids)
    return sentence

In [42]:
for _ in range(5):
    print(generate_sentence(model, max_len=100))

前作詩人が大合ったス古典学派の技術産業に対する援助である。このほか、同年11月に同法のことが分か国で、教会関係者の歴史において専門士や顎法制度及び審査研究所として欧州農地が設立した時代は対制度に基づく市内で導入されたため、双方の結合の元はされていない。このほか、32件では比較的前部しシリンダー39面のレベルにある。メタンを下ように、「ホッホは
1981年、工業規格施設は解散型で完了する。2011年7月17日にドコンボサーには、同年8月3日から2日トロデート日にはリーグトップと対戦。2007年のリフィ死者4には最後で出場した。
永エリアは毛里田地区の県有もアフリカ系サービスが館設定されている。終了後、阪急車両に導入が可能の臨時である。中立協同組合を続けていたために小さな町三丁目、7月29日公には装備欧米が本格2004年頃に編成する。
低価格は実際に多くのテレビと同様に以来本種国は猪木の商品と栽培されたホールであるターは9月には、などのオンエアツ国立公園総合に変更した。
適切な「合成経路」が論理であって出ている。また、「Opera MockG」と言えば商品展開し、リセットカップで初めて監督は終了することとなった。2013年シーズン終了後以降は2004年3組で、ドライバーズの半分でケモエターが当該スタジオ、ピクレジットと紹介し、1100万円の成績持病の割回を報道し、雨の先頭成績で、2012年9月、FA


マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減ったと思われる。